In [16]:
# modules we'll use
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# read in all our data
ks_projects_orig  = pd.read_csv("../src/ks-projects-201801.csv")

# set seed for reproducibility
np.random.seed(0) 

# look at a few rows of the ks_projects file
ks_projects_orig.sample(5)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
338862,796196901,10G Christmas Tree,Art,Art,USD,2010-12-26,10526.0,2010-12-08 08:44:04,0.0,failed,0,US,0.0,0.0,10526.0
277871,483825010,Gliff,Gaming Hardware,Games,USD,2016-03-28,10000.0,2016-01-28 04:56:18,51.0,failed,5,US,51.0,51.0,10000.0
47000,123916947,STUFFED Food Truck,Food Trucks,Food,USD,2015-01-06,60000.0,2014-11-07 02:24:36,25.0,failed,1,US,25.0,25.0,60000.0
111338,1565733636,NeoExodus Adventure: Origin of Man for Pathfin...,Tabletop Games,Games,USD,2012-05-01,500.0,2012-03-15 01:16:10,585.0,successful,17,US,585.0,585.0,500.0
53743,1273544891,NAPOLEON IN NEW YORK! an original TV Series,Comedy,Film & Video,USD,2016-07-26,25000.0,2016-05-27 00:07:25,25.0,failed,1,US,25.0,25.0,25000.0


In [17]:
# get info about DataFrame columns
ks_projects_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [18]:
# convert 'deadline' and 'launched' columns to datetime
ks_projects_orig['deadline'] = pd.to_datetime(ks_projects_orig['deadline'])
ks_projects_orig['launched'] = pd.to_datetime(ks_projects_orig['launched'])

# get info about DataFrame columns
ks_projects_orig.info()

# look at a few rows of the ks_projects file
ks_projects_orig.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                378661 non-null  int64         
 1   name              378657 non-null  object        
 2   category          378661 non-null  object        
 3   main_category     378661 non-null  object        
 4   currency          378661 non-null  object        
 5   deadline          378661 non-null  datetime64[ns]
 6   goal              378661 non-null  float64       
 7   launched          378661 non-null  datetime64[ns]
 8   pledged           378661 non-null  float64       
 9   state             378661 non-null  object        
 10  backers           378661 non-null  int64         
 11  country           378661 non-null  object        
 12  usd pledged       374864 non-null  float64       
 13  usd_pledged_real  378661 non-null  float64       
 14  usd_

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
299667,595962034,Button Poetry Live!,Mixed Media,Art,USD,2015-09-18,10000.0,2015-08-19 19:34:20,18216.27,successful,455,US,18216.27,18216.27,10000.00
181674,1924707671,"C STREET 2012 : Tbilisi, Georgia",World Music,Music,USD,2012-06-07,5000.0,2012-05-08 18:22:59,7210.69,successful,82,US,7210.69,7210.69,5000.00
137583,1698707842,Dérive's Next Project,Punk,Music,USD,2014-07-06,1200.0,2014-06-08 17:58:37,1255.66,successful,33,US,1255.66,1255.66,1200.00
296861,581269566,Photo Book - World Santa Claus Congress,Photobooks,Photography,DKK,2017-04-14,110000.0,2017-03-14 23:45:35,462.00,failed,5,DK,0.00,66.46,15823.47
66362,1337585114,Kickstart CLE Brewing to greatness!,Drinks,Food,USD,2017-08-19,6500.0,2017-07-20 21:22:43,250.00,failed,5,US,75.00,250.00,6500.00


In [19]:
# convert non numeric to numeric
le = LabelEncoder()
ks_projects_orig['category'] = le.fit_transform(ks_projects_orig['category'])
ks_projects_orig['main_category'] = le.fit_transform(ks_projects_orig['main_category'])
ks_projects_orig['state'] = le.fit_transform(ks_projects_orig['state'])
ks_projects_orig['country'] = le.fit_transform(ks_projects_orig['country'])

# get info about DataFrame columns
ks_projects_orig.info()

# look at a few rows of the ks_projects file
ks_projects_orig.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                378661 non-null  int64         
 1   name              378657 non-null  object        
 2   category          378661 non-null  int32         
 3   main_category     378661 non-null  int32         
 4   currency          378661 non-null  object        
 5   deadline          378661 non-null  datetime64[ns]
 6   goal              378661 non-null  float64       
 7   launched          378661 non-null  datetime64[ns]
 8   pledged           378661 non-null  float64       
 9   state             378661 non-null  int32         
 10  backers           378661 non-null  int64         
 11  country           378661 non-null  int32         
 12  usd pledged       374864 non-null  float64       
 13  usd_pledged_real  378661 non-null  float64       
 14  usd_

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
284129,515317667,Three Zombies in Philadelphia,55,6,USD,2013-05-29,20000.0,2013-03-30 19:36:04,29.99,1,4,22,29.99,29.99,20000.0
187025,1952141049,Chroma Cafe and Bakery Hood and Expansion Project,123,7,USD,2014-05-30,9750.0,2014-04-25 03:13:43,9900.00,3,134,22,9900.00,9900.00,9750.0
231103,24476326,The L Train,24,6,USD,2017-09-09,6000.0,2017-08-25 00:00:26,6065.00,3,81,22,340.00,6065.00,6000.0
117045,1594525425,Slimo - Desktop Buddy,131,13,USD,2017-11-01,250.0,2017-10-02 20:10:09,22.00,1,3,22,10.00,22.00,250.0
214293,2091909738,TheBoutSheet.Com Mobile Aps,151,13,USD,2015-06-24,25000.0,2015-05-18 23:40:29,0.00,1,0,22,0.00,0.00,25000.0


In [22]:
# drop redundant columns
ks_projects_reduced = ks_projects_orig.drop(['ID', 'name', 'currency', 'goal', 'pledged', 'usd pledged'], axis=1)

# get info about DataFrame columns
ks_projects_reduced.info()

# look at a few rows of the ks_projects file
ks_projects_reduced.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   category          378661 non-null  int32         
 1   main_category     378661 non-null  int32         
 2   deadline          378661 non-null  datetime64[ns]
 3   launched          378661 non-null  datetime64[ns]
 4   state             378661 non-null  int32         
 5   backers           378661 non-null  int64         
 6   country           378661 non-null  int32         
 7   usd_pledged_real  378661 non-null  float64       
 8   usd_goal_real     378661 non-null  float64       
dtypes: datetime64[ns](2), float64(2), int32(4), int64(1)
memory usage: 20.2 MB


,category,main_category,deadline,launched,state,backers,country,usd_pledged_real,usd_goal_real
146996,53,6,2017-12-28,2017-11-28 05:57:28,3,34,22,9209.00,7000.00
263453,10,0,2015-05-08,2015-04-08 00:28:05,3,49,9,2042.29,1849.63
315751,93,6,2011-05-26,2011-04-26 18:50:25,3,888,22,82468.55,80000.00
109139,2,5,2017-08-20,2017-07-21 17:00:44,1,13,22,258.00,406.00
275234,38,0,2016-06-09,2016-05-25 13:42:35,3,9,22,256.00,200.00


In [23]:
# convert to numpy_array
ks_projects_numpy = ks_projects_reduced.to_numpy()

# print first 5 rows
ks_projects_numpy[0:4, :]



array([[108, 12, Timestamp('2015-10-09 00:00:00'),
        Timestamp('2015-08-11 12:12:28'), 1, 0, 9, 0.0, 1533.95],
       [93, 6, Timestamp('2017-11-01 00:00:00'),
        Timestamp('2017-09-02 04:43:57'), 1, 15, 22, 2421.0, 30000.0],
       [93, 6, Timestamp('2013-02-26 00:00:00'),
        Timestamp('2013-01-12 00:20:50'), 1, 3, 22, 220.0, 45000.0],
       [90, 10, Timestamp('2012-04-16 00:00:00'),
        Timestamp('2012-03-17 03:24:11'), 1, 1, 22, 1.0, 5000.0]],
      dtype=object)